In [1]:
import os
import librosa
import numpy as np
import soundfile as sf
from tqdm import tqdm

# ✅ Set paths
soundscape_dir = "E:/birdclef-2024/unlabeled_soundscapes"
output_dir = "E:/birdclef-2024/unlabeled_segments_filtered"
os.makedirs(output_dir, exist_ok=True)

# ✅ Parameters
sample_rate = 32000
segment_duration = 5  # seconds
samples_per_segment = sample_rate * segment_duration
max_segments_per_file = 10

# ✅ Process each soundscape file
for filename in tqdm(os.listdir(soundscape_dir)):
    if not filename.endswith(".ogg") and not filename.endswith(".wav"):
        continue

    file_path = os.path.join(soundscape_dir, filename)
    y, sr = librosa.load(file_path, sr=sample_rate)
    soundscape_id = os.path.splitext(filename)[0]

    # Detect non-silent intervals (in samples)
    intervals = librosa.effects.split(y, top_db=30)  # adjust top_db to tune sensitivity

    segment_count = 0
    for start_sample, end_sample in intervals:
        if segment_count >= max_segments_per_file:
            break

        segment = y[start_sample:end_sample]

        # Skip if the active segment is shorter than 5 seconds
        if len(segment) < samples_per_segment:
            continue

        # Chop into consecutive 5s chunks from this active region
        for i in range(0, len(segment) - samples_per_segment + 1, samples_per_segment):
            if segment_count >= max_segments_per_file:
                break

            chunk = segment[i:i + samples_per_segment]
            segment_filename = f"{soundscape_id}_{segment_count*5}_{(segment_count+1)*5}.wav"
            segment_path = os.path.join(output_dir, segment_filename)
            sf.write(segment_path, chunk, samplerate=sample_rate)
            segment_count += 1

print("✅ Filtered soundscape segmentation completed.")


100%|██████████| 8444/8444 [58:12<00:00,  2.42it/s] 

✅ Filtered soundscape segmentation completed.


In [5]:
import os
import librosa
import numpy as np
import cv2
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
from matplotlib import cm

# ✅ Paths
input_dir = "E:/birdclef-2024/unlabeled_segments_filtered"
output_dir = "E:/birdclef-2024/unlabeled_mels"
os.makedirs(output_dir, exist_ok=True)

# ✅ Parameters
sample_rate = 32000
n_mels = 128
n_fft = 1042
hop_length = 500
batch_size = 1000

# ✅ Get list of .wav files
all_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

for batch_start in range(0, len(all_files), batch_size):
    batch_files = all_files[batch_start:batch_start + batch_size]

    for filename in tqdm(batch_files, desc=f"Processing batch {batch_start // batch_size + 1}"):
        file_path = os.path.join(input_dir, filename)
        y, sr = librosa.load(file_path, sr=sample_rate)

        # Generate mel spectrogram in dB
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # Normalize and convert to colormap (like matplotlib's 'viridis')
        S_dB_norm = 255 * (S_dB - S_dB.min()) / (S_dB.max() - S_dB.min())
        S_img = S_dB_norm.astype(np.uint8)

        colormap = cm.get_cmap('viridis')
        S_colored = colormap(S_img / 255.0)[:, :, :3]  # Drop alpha channel
        S_rgb = (S_colored * 255).astype(np.uint8)

        # Resize to 224x224
        S_img_resized = cv2.resize(S_rgb, (224, 224), interpolation=cv2.INTER_AREA)

        # Save as PNG image
        output_path = os.path.join(output_dir, filename.replace(".wav", ".png"))
        cv2.imwrite(output_path, cv2.cvtColor(S_img_resized, cv2.COLOR_RGB2BGR))

        gc.collect()

    print(f"✅ Completed batch {batch_start // batch_size + 1}")

print("✅ Mel spectrogram generation completed in color with colormap.")


Processing batch 1:   0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\ONGC\AppData\Local\Temp\ipykernel_40168\35302409.py:40: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('viridis')
Processing batch 1: 100%|██████████| 1000/1000 [03:51<00:00,  4.33it/s]


✅ Completed batch 1


Processing batch 2: 100%|██████████| 1000/1000 [03:49<00:00,  4.36it/s]


✅ Completed batch 2


Processing batch 3: 100%|██████████| 1000/1000 [03:40<00:00,  4.53it/s]


...

✅ Completed batch 80


Processing batch 81: 100%|██████████| 1000/1000 [03:46<00:00,  4.41it/s]


✅ Completed batch 81


Processing batch 82: 100%|██████████| 279/279 [01:04<00:00,  4.35it/s]

✅ Completed batch 82
✅ Mel spectrogram generation completed in color with colormap.


In [2]:
#INFERENCE MODEL ON UNLABELED SOUNDSCAPE USING RESNET-50

import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np

# ✅ Paths
model_path = "best_resnet50_model.pth"
mels_dir = "E:/birdclef-2024/unlabeled_mels"
output_csv = "E:/birdclef-2024/unlabeled_predictions.csv"

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Load model
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 181)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ✅ Transforms
inference_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# ✅ Check existing predictions to resume
existing_files = set()
if os.path.exists(output_csv):
    existing_df = pd.read_csv(output_csv)
    existing_files = set(existing_df['filename'].tolist())
    results = existing_df.values.tolist()
else:
    results = []

# ✅ Inference
image_files = sorted([f for f in os.listdir(mels_dir) if f.endswith(".png")])
batch_size = 1000  # Save progress every 1000 files
buffer = []

with torch.no_grad():
    for idx, img_file in enumerate(tqdm(image_files)):
        if img_file in existing_files:
            continue

        img_path = os.path.join(mels_dir, img_file)
        image = Image.open(img_path).convert("RGB")
        tensor = inference_transforms(image).unsqueeze(0).to(device)

        outputs = model(tensor)
        probs = torch.softmax(outputs, dim=1).squeeze().cpu().numpy()

        buffer.append([img_file] + list(probs))

        # Save every `batch_size` predictions
        if len(buffer) >= batch_size:
            results.extend(buffer)
            df = pd.DataFrame(results, columns=["filename"] + [f"class_{i}" for i in range(181)])
            df.to_csv(output_csv, index=False)
            buffer = []  # Clear buffer

# ✅ Final save
if buffer:
    results.extend(buffer)
    df = pd.DataFrame(results, columns=["filename"] + [f"class_{i}" for i in range(181)])
    df.to_csv(output_csv, index=False)

print("✅ Inference completed and predictions saved.")


Using device: cpu


C:\Users\ONGC\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ONGC\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 81279/81279 [00:00<00:00, 2881828.93it/s]

✅ Inference completed and predictions saved.


In [4]:
#INFERENCE MODEL ON UNLABELED SOUNDSCAPE USING EFFICIENT NET-B3

import os
import torch
import torch.nn as nn
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np
import timm
from torchvision import transforms

# ✅ Paths
model_path = "best_efficientnet_b3_model.pth"
mels_dir = "E:/birdclef-2024/unlabeled_mels"
output_csv = "E:/birdclef-2024/unlabeled_predictions_b3.csv"

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Load EfficientNet-B3 model
model = timm.create_model("efficientnet_b3", pretrained=False, num_classes=181)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ✅ Transforms (300x300 for EfficientNet-B3)
inference_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# ✅ Check existing predictions to resume
existing_files = set()
if os.path.exists(output_csv):
    existing_df = pd.read_csv(output_csv)
    existing_files = set(existing_df['filename'].tolist())
    results = existing_df.values.tolist()
else:
    results = []

# ✅ Inference loop
image_files = sorted([f for f in os.listdir(mels_dir) if f.endswith(".png")])
batch_size = 1000
buffer = []

with torch.no_grad():
    for idx, img_file in enumerate(tqdm(image_files)):
        if img_file in existing_files:
            continue

        img_path = os.path.join(mels_dir, img_file)
        image = Image.open(img_path).convert("RGB")
        tensor = inference_transforms(image).unsqueeze(0).to(device)

        outputs = model(tensor)
        probs = torch.softmax(outputs, dim=1).squeeze().cpu().numpy()

        buffer.append([img_file] + list(probs))

        if len(buffer) >= batch_size:
            results.extend(buffer)
            df = pd.DataFrame(results, columns=["filename"] + [f"class_{i}" for i in range(181)])
            df.to_csv(output_csv, index=False)
            buffer = []

# ✅ Final save
if buffer:
    results.extend(buffer)
    df = pd.DataFrame(results, columns=["filename"] + [f"class_{i}" for i in range(181)])
    df.to_csv(output_csv, index=False)

print("✅ Inference completed and predictions saved.")


Using device: cpu


100%|██████████| 81279/81279 [1:39:04<00:00, 13.67it/s]  


✅ Inference completed and predictions saved.
